In [1]:
import pandas as pd
import altair as alt

from vega_datasets import data


alt.data_transformers.disable_max_rows()

df = pd.read_csv('EU-UK-Weather-Data.csv')
df.columns

Index(['country', 'location_name', 'latitude', 'longitude',
       'temperature_celsius', 'humidity', 'air_quality_Ozone', 'date', 'day',
       'month', 'year', 'week', 'time', 'season', 'condition'],
      dtype='object')

In [2]:
HEIGHT, WIDTH = 900, 500

source = alt.topo_feature(data.world_110m.url, 'countries')
date_range = (-10, 0)
map_brush = alt.selection_interval(value={'longitude': [-3, 3], 'latitude': [50, 55]})  # this crops the chart
map_brush = alt.selection_interval()
line_brush = alt.selection_interval()

base = alt.layer(
    alt.Chart(source).mark_geoshape(fill='lightgray', stroke='gray')
).properties(
    width=WIDTH,
    height=HEIGHT,
    projection={
        "type": "azimuthalEqualArea",        
        "rotate": [-10, -52],
        "translate": [WIDTH//2, HEIGHT//2],
        "scale": 700,
        "precision": 0.1
})
city_points = alt.Chart(df).mark_circle(size=50).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    color=alt.Color('location_name:N', title="City"),
    opacity=alt.condition(map_brush, alt.value(1), alt.value(0.2)),
    tooltip=['location_name:N', 'country:N']
).properties(
    width=WIDTH,
    height=HEIGHT
).add_params(
    map_brush
).transform_filter(
    line_brush
)
map_view = base + city_points

temp_line = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('temperature_celsius:Q', title='Temperature'),
    color='location_name:N',
    shape='condition:N',
    tooltip=['date:T', 'location_name:N', 'temperature_celsius:Q']
).properties(
    width=WIDTH, height=HEIGHT // 3
).add_params(
    line_brush
).transform_filter(
    map_brush
)
humidity_line = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('humidity:Q', title='Humidity'),
    color='location_name:N',
    shape='condition:N',
    tooltip=['date:T', 'location_name:N', 'humidity:Q']
).properties(
    width=WIDTH, height=HEIGHT // 3
).transform_filter(
    map_brush,
    line_brush
)
air_quality_line = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('air_quality_Ozone:Q', title='Air Quality Ozone'),
    color='location_name:N',
    shape='condition:N',
    tooltip=['date:T', 'location_name:N', 'air_quality_Ozone:Q']
).properties(
    width=WIDTH, height=HEIGHT // 3
).transform_filter(
    map_brush,
    line_brush
)

system1 = map_view | (temp_line & humidity_line & air_quality_line)
system1.save('system1.html')
system1

alt.HConcatChart(...)

* System 1: Map View + 3 Line Charts (1 for temp, humidity, AQI)
* System 2: Map View + 3 Bar Charts (1 for temp, humidity, AQI)
* System 3: City Dropdown + Heatmap 

## Tasks
1. Identify the week # that had the hottest temperatures in London, and what was the temperature.
2. Find 3 cities with ideal conditions (temp: 22 to 24, humidity: 40 to 50) in October.
3. Which city had the most humidity overall?
4. Determine the season when air quality was the best in Madrid.
5. Which city had the most sunny days in September 2024.